In [2]:
import pandas as pd
data =pd.read_csv('/content/LoadDatainkW.csv')

In [3]:
data.head(34)
data.shape

(2184, 10)

In [4]:
day_1 = data.iloc[0:-24, 2]
day_2 = data.iloc[24:, 2]
print(day_1.shape)
print(day_2.shape)

(2160,)
(2160,)


In [6]:
day_2 = day_2.reset_index()
day_2 = day_2['Load (kW)']

In [7]:
df = pd.concat([day_1, day_2], axis = 1)
df.shape

(2160, 2)

In [8]:
df.columns = ['Day_1', 'Day_2']
df.head

<bound method NDFrame.head of            Day_1       Day_2
0     5551.82208  4931.26380
1     4983.17184  4775.53968
2     4888.39680  4713.81120
3     5072.95872  4689.02628
4     5196.25980  4844.75040
...          ...         ...
2155  4643.97696  4626.51840
2156  5813.70048  4567.90752
2157  7115.92200  4723.63164
2158  6189.99480  5150.27520
2159  5657.97636  6938.37468

[2160 rows x 2 columns]>

In [9]:
normalized_data = (df-df.mean())/df.std()
normalized_data.head()

,Day_1,Day_2
0,-0.454862,-1.026890
1,-0.988072,-1.172657
2,-1.076940,-1.230438
3,-0.903881,-1.253638
4,-0.788264,-1.107871


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(normalized_data.Day_1, normalized_data.Day_2, test_size = 0.10, random_state = 42)

In [12]:
m = 1
c = -1
lr = 0.01
delta_m = 1 #Initialising Δm
delta_c = 1 #Initialising Δc
v_m = 0
v_c = 0
lam = 0.9
max_iters = 100 #Maximum number of iterations  
iters_count = 0 #Counting Iterations


def deriv(m_f, c_f, x, y):
  m_deriv = -1*(y-m_f*x-c_f)*x
  c_deriv = -1*(y-m_f*x-c_f)
  return m_deriv, c_deriv  


while iters_count < max_iters:
  for i in range(x_train.shape[0]):
    delta_m, delta_c = deriv(m, c, x_train.iloc[i], y_train.iloc[i])
    v_m = lam * v_m - lr * delta_m
    v_c = lam * v_c - lr * delta_c
    m += v_m
    c += v_c
  iters_count += 1
  print(f"Iteration: {iters_count}\tValue of m: {m}, \tValue of c: {c}")

print(f"\nThe local minima occurs at: {m}, {c}")

Iteration: 1	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 2	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 3	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 4	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 5	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 6	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 7	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 8	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 9	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 10	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 11	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 12	Value of m: 0.8885266666378754, 	Value of c: -0.015125603603097632
Iteration: 13	Value of m: 0.888526666

In [13]:
import numpy as np

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [14]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

In [15]:
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [16]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
mse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.6888134336509062
Mean square error 0.4744639463779513
Mean absolute error 0.4922787663231915


In [17]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.6650642307346853
Mean square error 0.4423104310027188
Mean absolute error 0.48048030390084917


In [21]:
hour = input("Enter an hour(0-23) of 1st December, 2018 to predict the load")
index = 2137 + int(hour)
x = normalized_data.iloc[index, 1]
normalised_output = m * x + c
output = (normalised_output * df.std()) + df.mean()
print(f"Predicted Load (kW) at {hour} hours on 1st December, 2018: {output[0]}")

Enter an hour(0-23) of 1st December, 2018 to predict the load9
Predicted Load (kW) at 9 hours on 1st December, 2018: 5692.727136365451
